# Custom ConvNet

Training a custom model from scratch takes too much time, thus I'll not try any hyperparameter tunig here.

### imports

In [13]:
import numpy as np
import tensorflow as tf
from PIL import Image

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D, Flatten

import matplotlib.pyplot as plt


### Load and augment images

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     rotation_range=5,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(200, 200),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                    target_size=(200, 200),
                                                    batch_size=32,
                                                    class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Model

In [7]:
class XrayModel(tf.keras.Model):
    def __init__(self):
        super(XrayModel, self).__init__()

        self.conv1 = Conv2D(16, kernel_size=5, padding='same', activation='relu')
        self.conv2 = Conv2D(32, kernel_size=5, padding='same', activation='relu')
        self.conv3 = Conv2D(64, kernel_size=5, padding='same', activation='relu')
        self.conv4 = Conv2D(128, kernel_size=5, padding='same', activation='relu')
        self.conv5 = Conv2D(256, kernel_size=5, padding='same', activation='relu')

        self.maxPool = MaxPool2D(pool_size=(2, 2))

        self.flatten = Flatten()
        self.dense = Dense(256, activation='relu')
        self.classifier = Dense(3, activation='softmax')
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.maxPool(x)

        x = self.conv2(x)
        x = self.maxPool(x)

        x = self.conv3(x)
        x = self.maxPool(x)

        x = self.conv4(x)
        x = self.maxPool(x)

        x = self.conv5(x)
        x = self.maxPool(x)

        x = self.flatten(x)
        x = self.dense(x)
        outputs = self.classifier(x)
        return outputs
        
        

### Compile and train

In [8]:
model = XrayModel()

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics='accuracy')

In [10]:
model.fit(train_generator,
         steps_per_epoch=160,
         epochs=20,
         validation_data=test_generator,
         validation_steps=800)

Epoch 1/20
160/160 [==============================] - 250s 2s/step - loss: 0.3912 - accuracy: 0.8419 - val_loss: 0.2617 - val_accuracy: 0.9115
Epoch 2/20
160/160 [==============================] - 216s 1s/step - loss: 0.3235 - accuracy: 0.8719
Epoch 3/20
160/160 [==============================] - 215s 1s/step - loss: 0.2667 - accuracy: 0.8998
Epoch 4/20
160/160 [==============================] - 215s 1s/step - loss: 0.2424 - accuracy: 0.9079
Epoch 5/20
160/160 [==============================] - 228s 1s/step - loss: 0.2105 - accuracy: 0.9216
Epoch 6/20
160/160 [==============================] - 207s 1s/step - loss: 0.1759 - accuracy: 0.9372
Epoch 7/20
160/160 [==============================] - 215s 1s/step - loss: 0.1730 - accuracy: 0.9372
Epoch 8/20
160/160 [==============================] - 216s 1s/step - loss: 0.1670 - accuracy: 0.9427
Epoch 9/20
160/160 [==============================] - 216s 1s/step - loss: 0.1602 - accuracy: 0.9437
Epoch 10/20
160/160 [============================

## Evaluate

In [11]:
model.evaluate(test_generator)

41/41 [==============================] - 34s 829ms/step - loss: 0.1249 - accuracy: 0.9503


[0.12490388751029968, 0.9503105878829956]

The default model managed to obtain over 95% accuracy of the cross-validation set.